### 0. Install and load library

In [ ]:
#Restart kernel after installation: Runtime -> Restart runtime

#!pip install -U sentencepiece datasets
#!pip install -U transformers sentencepiece datasets, seqeval

In [2]:
import transformers

### 1. Fine-tuning DistillBERT model on CONLL2003 NER dataset.

In [1]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/tensorflow/token_classification/run_ner.py 

--2021-07-10 04:25:33--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/tensorflow/token_classification/run_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22263 (22K) [text/plain]
Saving to: ‘run_ner.py’

run_ner.py          100%[===================>]  21.74K  --.-KB/s    in 0s      

2021-07-10 04:25:33 (112 MB/s) - ‘run_ner.py’ saved [22263/22263]



In [6]:
!python /content/run_ner.py \
  --model_name_or_path distilbert-base-cased \
  --label_all_tokens True \
  --return_entity_level_metrics True \
  --dataset_name conll2003 \
  --output_dir /tmp/distilbert-base-cased-finetuned-conll03-english_tf \
  --do_train \
  --do_eval \
  --do_predict \
  --evaluation_strategy epoch \
  --save_strategy epoch \
  --logging_strategy epoch 

2021-07-10 04:29:24.864148: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)
loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
   

In [ ]:
"""
Evaluation metrics:
LOC_precision: 0.9633
LOC_recall: 0.9521
LOC_f1: 0.9577
LOC_number: 3635.0000
MISC_precision: 0.8486
MISC_recall: 0.8784
MISC_f1: 0.8632
MISC_number: 1480.0000
ORG_precision: 0.8958
ORG_recall: 0.9127
ORG_f1: 0.9041
ORG_number: 2702.0000
PER_precision: 0.9514
PER_recall: 0.9588
PER_f1: 0.9551
PER_number: 3329.0000
overall_precision: 0.9275
overall_recall: 0.9348
overall_f1: 0.9311
overall_accuracy: 0.9829
Configuration saved in /tmp/distilbert-base-cased-finetuned-conll03-english_tf/config.json
Model weights saved in /tmp/distilbert-base-cased-finetuned-conll03-english_tf/tf_model.h5
"""

### 2. Inference using pipeline

In [8]:
!ls /tmp/distilbert-base-cased-finetuned-conll03-english_tf

config.json  tf_model.h5


In [ ]:
# Update NER label2id and id2label in model config file: config.json
"""
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "i-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },


  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },

"""

In [ ]:
from transformers import pipeline
distilbert_ner = pipeline('ner', model="/tmp/distilbert-base-cased-finetuned-conll03-english_tf/", tokenizer="distilbert-base-cased", aggregation_strategy='first')

In [17]:
article = """
KUALA LUMPUR - Malaysian Prime Minister Muhyiddin Yassin's party said on Thursday (July 8) that his government would continue to function despite Umno withdrawing its backing. 
Amid uncertainty over whether Tan Sri Muhyiddin continues to command majority support without Umno, the largest party in the Perikatan Nasional (PN) ruling pact, 
his Parti Pribumi Bersatu Malaysia said Umno's decision "had no effect on the workings of government"."""


results = distilbert_ner(article)

In [19]:
print("Predicted:")
for tag in results:
    print(f"{tag['entity_group']:<5} as {tag['word']}")


"""
Output
------
Predicted:
ORG   as KUALA LUMPUR
MISC  as Malaysian
PER   as Muhyiddin
PER   as Yassin
PER   as Umno
PER   as Tan
PER   as Sri
PER   as Muhyiddin
PER   as Umno
ORG   as Perikatan Nasional
ORG   as PN
ORG   as Parti Pribumi Bersatu Malaysia
PER   as Umno

"""

Predicted:
ORG   as KUALA LUMPUR
MISC  as Malaysian
PER   as Muhyiddin
PER   as Yassin
PER   as Umno
PER   as Tan
PER   as Sri
PER   as Muhyiddin
PER   as Umno
ORG   as Perikatan Nasional
ORG   as PN
ORG   as Parti Pribumi Bersatu Malaysia
PER   as Umno


'\nOutput\n------\nPredicted:\nORG   as KUALA LUMPUR\nMISC  as Malaysian\nPER   as Muhyiddin\nPER   as Yassin\nPER   as Umno\nPER   as Tan\nPER   as Sri\nPER   as Muhyiddin\nPER   as Umno\nORG   as Perikatan Nasional\nORG   as PN\nORG   as Parti Pribumi Bersatu Malaysia\nPER   as Umno\n\n'